In [33]:
import pandas as pd
from scipy.stats import hypergeom

def analizar_expresion_genica(csv_file, filtros_grupo1, filtros_grupo2, output_prefix):
    # Cargar datos
    df = pd.read_csv(csv_file, index_col=0)  # Asume que la primera columna es el índice
    
    # Definir las columnas de metadatos
    columnas_metadatos = ["cell_barcode", "gene_count", "umi_count", "doublet_score", "x", "y", 
                          "cluster_alias", "cell_in_wmb_study", "wmb_cluster_alias", "library_label", "alignment_job_id", 
                          "library_method", "barcoded_cell_sample_label", "enrichment_population", "region_of_interest_label", 
                          "anatomical_division_label", "library_in_wmb_study", "donor_label", "population_sampling", 
                          "donor_genotype", "donor_sex", "donor_age", "donor_age_category", "donor_in_wmb_study", 
                          "feature_matrix_label", "dataset_label", "abc_sample_id", "neurotransmitter_combined_label", "cluster_name"]
    
    # Filtrar las columnas de expresión
    columnas_expresion = [col for col in df.columns if col not in columnas_metadatos]
    df_expresion = df[columnas_expresion]
    
    # Filtrar las células según los criterios de metadatos
    df_filtros = df[columnas_metadatos]
    
    def aplicar_filtros(df, filtros):
        """Aplica múltiples filtros de metadatos y devuelve las columnas correspondientes."""
        filtro_general = pd.Series(True, index=df.index)
        for col, valor in filtros.items():
            if col in df.columns:
                filtro_general &= df[col] == valor
            else:
                print(f"Advertencia: La columna {col} no está en los datos")
        return df[filtro_general].index
    
    celdas_grupo1 = aplicar_filtros(df_filtros, filtros_grupo1)
    celdas_grupo2 = aplicar_filtros(df_filtros, filtros_grupo2)
    
    # Filtrar las columnas de expresión según las células seleccionadas
    grupo1 = df_expresion.loc[celdas_grupo1]
    grupo2 = df_expresion.loc[celdas_grupo2]
    
    # Calcular la expresión media por gen en cada grupo
    media_grupo1 = grupo1.mean(axis=0)
    media_grupo2 = grupo2.mean(axis=0)
    
    # Genes únicos según tu definición
    genes_unicos_grupo1 = ((media_grupo1 / (media_grupo1 - media_grupo2)) > 1) & ((media_grupo1 / (media_grupo1 - media_grupo2)) < 3)
    genes_unicos_grupo2 = ((media_grupo2 / (media_grupo2 - media_grupo1)) > 1) & ((media_grupo2 / (media_grupo2 - media_grupo1)) < 3)
    
    df_unicos_grupo1 = pd.DataFrame({"Expresion_Grupo1": media_grupo1[genes_unicos_grupo1]})
    df_unicos_grupo2 = pd.DataFrame({"Expresion_Grupo2": media_grupo2[genes_unicos_grupo2]})
    
    # Genes sobreexpresados
    genes_sobreexpresados_grupo1 = media_grupo1 > 0.5
    genes_sobreexpresados_grupo2 = media_grupo2 > 0.5
    
    df_sobreexpresados_grupo1 = pd.DataFrame({"Expresion_Grupo1": media_grupo1[genes_sobreexpresados_grupo1]})
    df_sobreexpresados_grupo2 = pd.DataFrame({"Expresion_Grupo2": media_grupo2[genes_sobreexpresados_grupo2]})

    # Convertir filtros en nombres de archivo
    def filtros_a_nombre(filtros):
        return "_".join([f"{k}-{v}" for k, v in filtros.items()]).replace(" ", "_")

    nombre_grupo1 = filtros_a_nombre(filtros_grupo1)
    nombre_grupo2 = filtros_a_nombre(filtros_grupo2)
    
    ruta_base = "G:/Alberto/Transcriptomics/Results/"

    # Guardar archivos CSV
    df_unicos_grupo1.to_csv(f"{ruta_base}{output_prefix}_genes_unicos_{nombre_grupo1}.csv")
    df_unicos_grupo2.to_csv(f"{ruta_base}{output_prefix}_genes_unicos_{nombre_grupo2}.csv")
    df_sobreexpresados_grupo1.to_csv(f"{ruta_base}{output_prefix}_genes_sobreexpresados_{nombre_grupo1}.csv")
    df_sobreexpresados_grupo2.to_csv(f"{ruta_base}{output_prefix}_genes_sobreexpresados_{nombre_grupo2}.csv")
    
    # Guardar archivos para PANTHER
    def guardar_panther(df, filename):
        with open(f"{ruta_base}{filename}", 'w') as file:
            for gene, expresion in df.iloc[:, 0].items():
                file.write(f"{gene}\t{expresion}\n")

    guardar_panther(df_unicos_grupo1, f"{output_prefix}_genes_unicos_{nombre_grupo1}_panther.txt")
    guardar_panther(df_unicos_grupo2, f"{output_prefix}_genes_unicos_{nombre_grupo2}_panther.txt")
    guardar_panther(df_sobreexpresados_grupo1, f"{output_prefix}_genes_sobreexpresados_{nombre_grupo1}_panther.txt")
    guardar_panther(df_sobreexpresados_grupo2, f"{output_prefix}_genes_sobreexpresados_{nombre_grupo2}_panther.txt")

    print("Análisis completado. Archivos guardados.")

# Ejemplo de uso:
archivo_csv = r"G:\Alberto\Transcriptomics\Results\cells_with_genes_allgene.csv"

# Comparar por edad
filtros_grupo1 = {"donor_age_category": "aged"}
filtros_grupo2 = {"donor_age_category": "adult"}
analizar_expresion_genica(archivo_csv, filtros_grupo1, filtros_grupo2, output_prefix="resultado_edad")

# Comparar por sexo
#filtros_grupo1 = {"donor_sex": "M"}
#filtros_grupo2 = {"donor_sex": "F"}
#analizar_expresion_genica(archivo_csv, filtros_grupo1, filtros_grupo2, output_prefix="resultado_sexo")

# Comparar por método de secuenciación
#filtros_grupo1 = {"library_method": "Method_A"}
#filtros_grupo2 = {"library_method": "Method_B"}
#analizar_expresion_genica(archivo_csv, filtros_grupo1, filtros_grupo2, output_prefix="resultado_metodo")

# Comparar por edad y sexo
#filtros_grupo1 = {"donor_age_category": "Young", "donor_sex": "M"}
#filtros_grupo2 = {"donor_age_category": "Adult", "donor_sex": "F"}
#analizar_expresion_genica(archivo_csv, filtros_grupo1, filtros_grupo2, output_prefix="resultado_edad_sexo")


Análisis completado. Archivos guardados.
